In [ ]:
# pip install boto3 --force-reinstall --quiet

In [ ]:
# %pip install ragas==0.0.20 --force-reinstall --quiet

In [1]:
# pip install -r "/home/ec2-user/SageMaker/AscendNotebook/ascendrequirements.txt"

In [2]:
# restart kernel
# from IPython.core.display import HTML
# HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [3]:
# import RetrieveNGenerate

from RetrieveNGenerate import RetrieveAndGenerate as RAG

# Import the os module for interacting with the operating system
import os

# Import the boto3 library for interacting with AWS services
import boto3

import pandas as pd

# Importing the datetime class from the datetime module to handle date and time objects
from datetime import datetime

# Importing the time module to handle time-related functions
import time

from langfuse import Langfuse

#import metrics
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall
)

from ragas.llms import LangchainLLM

from langchain_aws import ChatBedrock
from langchain.embeddings import BedrockEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.retrievers.bedrock import AmazonKnowledgeBasesRetriever
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

session Session(region_name='us-east-1')
AWS Region Name: us-east-1


In [4]:
model_id = "anthropic.claude-3-haiku-20240307-v1:0" # try with both claude instant as well as claude-v2. for claude v2 - "anthropic.claude-v2"
MDL_TYPE = 'ANTH_CLAUDE_Haiku_'
kb_id = "EJBFFIJDIJ"

# get keys for your project from https://cloud.langfuse.com
LANGFUSE_PUBLIC_KEY = "XXXXXXXXXXX" #replace it with your public key
LANGFUSE_SECRET_KEY = "XXXXXXXXXXX" #replace it with you secret key
LANGFUSE_HOST="https://us.cloud.langfuse.com"

MX_TKN = 4000
TMPRT = 0.1
TOP_P = 0.5
TOP_K = 1

bedrock_client = boto3.client('bedrock-runtime')

model_kwargs_claude = {
    "temperature": 0,
    "top_k": 0,
    "max_tokens": 4000
}

llm_for_text_generation = ChatBedrock(model_id=model_id,
              model_kwargs=model_kwargs_claude,
              streaming=True,
              client = bedrock_client,)


llm_for_evaluation = ChatBedrock(model_id=model_id,
              model_kwargs=model_kwargs_claude,
              streaming=True,
              client = bedrock_client,)

bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1",client=bedrock_client)

boto3_session = boto3.session.Session()
region_name = boto3_session.region_name

print("session", boto3_session)
print("AWS Region Name:", region_name)

session Session(region_name='us-east-1')
AWS Region Name: us-east-1


In [5]:
# The directory to save results
Rslts_Save_Dir = "/home/ec2-user/SageMaker/AscendNotebook/CodeExecutionMetrics/"

print("Present working directory:", os.getenv('PWD'))
print("Directory to save results:", Rslts_Save_Dir)

Present working directory: /home/ec2-user/SageMaker/AscendNotebook/Testing_RAG_LLMs
Directory to save results: /home/ec2-user/SageMaker/AscendNotebook/CodeExecutionMetrics/


In [6]:
retriever = AmazonKnowledgeBasesRetriever(
        knowledge_base_id=kb_id,
        retrieval_config={"vectorSearchConfiguration": {"numberOfResults": 4}},
        # endpoint_url=endpoint_url,
        # region_name="us-east-1",
        # credentials_profile_name="<profile_name>",
    )

In [7]:
PROMPT_TEMPLATE = """
    You are the best customer acquisition data analyst and strategist, you provide answers to questions by using fact based and statistical information when possible. 
    Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    <context>
    {context}
    </context>

    <question>
    {question}
    </question>

    The response should be specific and use statistics or numbers when possible.

    Assistant:"""

prompt = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)

retriever = AmazonKnowledgeBasesRetriever(
        knowledge_base_id=kb_id,
        retrieval_config={"vectorSearchConfiguration": {"numberOfResults": 4}},
        # endpoint_url=endpoint_url,
        # region_name="us-east-1",
        # credentials_profile_name="<profile_name>",
    )

# Setup RAG pipeline
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | llm_for_text_generation
    | StrOutputParser() 
)

In [8]:
questions = ["What products use nylon?", 
"give me a list of top 100 products that use nylon (10 is too small and is no representative)?", 
"what companies use Nylon in their products?", 
"who are the biggest users of nylon?", 
"who is compounding Nylon in North America and South America or Latin America?", 
"Are compounders using more recycled feedstock?", 
"What innovations are driving the use of recycled polypropylene?", 
"What are the main applications of recycled PP in the automotive industry?", 
"Which brands are known for their use of recycled PP in their products?", 
"What are some examples of high-volume products produced by leading companies using injection molding?", 
"How do aerospace companies utilize nylon 6,6 in their components?", 
"List all the companies extruding Nylon for extruded profiles and sheet?", 
"Which automotive companies are innovating with nylon 6,6 to enhance vehicle performance?", 
"In the automotive sector, what are the key components produced using nylon 6,6, and which manufacturers specialize in their production?", 
"What are the major industrial machinery components manufactured from nylon 6,6, and which companies are the leading suppliers of these components?", 
"Which consumer electronics companies rely on nylon 6,6 for the production of their casings and structural components, and how much nylon 6,6 do they consume annually?", 
"Can you list the flagship consumer electronic devices that feature nylon 6,6 components, and identify the manufacturers responsible for their production?", 
"In the automotive sector, what are the critical components manufactured from nylon 6,6, and which suppliers specialize in the production of these components?"]

In [9]:
# questions = ["Give me a list of 100 Injection Molders company names by injection molding sales and rank?",
#             "Give me the list of TOP Profile Extruders companies?",
#             "Give me the list of TOP THERMOFORMERS companies?",
#             "Give me a list of top 5 Injection Molders and their molding sales in $?"]


# questions = ["What products use nylon?", 
# "give me a list of top 100 products that use nylon (10 is too small and is no representative)?", 
# "what companies use Nylon in their products?", 
# "who are the biggest users of nylon?"]

In [10]:
# question = "Give me a list of 100 Injection Molders company names by injection molding sales and rank?"

In [11]:
langfuse = Langfuse(public_key=LANGFUSE_PUBLIC_KEY,secret_key=LANGFUSE_SECRET_KEY, host = LANGFUSE_HOST)
print("Check connection to Langfuse:", langfuse.auth_check())

Check connection to Langfuse: True


In [12]:
ragas_bedrock_model = LangchainLLM(llm_for_evaluation)

#set embeddings model for evaluating answer relevancy metric
answer_relevancy.embeddings = bedrock_embeddings

metrics = [
        faithfulness,
        answer_relevancy
    ]

for m in metrics:
    m.__setattr__("llm", ragas_bedrock_model)

In [13]:
def score_with_ragas(query, chunks, answer):
    scores = {}
    for m in metrics:
#         print(f"calculating {m.name}")
        scores[m.name] = m.score_single(
            {"question": query, "contexts": chunks, "answer": answer}
        )
    return scores

In [14]:
# counter = 1
# Q_A_DF = pd.DataFrame(columns = ['Question_Num', 'Question', 'Answer', 'Answer_Relevancy', 'faithfulness', 'context_sources', 'Contexts'])
# for query in questions:
#     Q_Num = "Question - " + str(counter)
#     print("question number:", Q_Num)
#     print("Question being processed: ",query)

#     question = query
#     Ans = "Answer - " + str(counter)
#     trace = langfuse.trace(name="RAG_W_512_Chunk", user_id = "Sai_Test", tags=["EXP 06/15/2024", Q_Num])
#     contexts = [docs.page_content for docs in retriever.get_relevant_documents(query)]
#     trace.span(name="retrieval", 
#                input={"question": question}, 
#                output={"contexts": contexts},
#         )
    
#     # Reterieve and Generate response based on question, model_type, temperature, etc.
#     response = RAG(TMPRT, MX_TKN, TOP_K, TOP_P, question, kb_id, region_name, '', model_id)
#     print(response)

#     # Used Context for Generating Answers
#     contxt = [reference['content']['text'] for citation in response['citations'] for reference in citation['retrievedReferences']]

#     # Generated Answers
#     generated_answer = response["output"]["text"]

#     # Context Source
#     context_source = [reference['location']['s3Location']['uri'] for citation in response['citations'] for reference in citation['retrievedReferences']]

# #     print(generated_answer)
# #     print(" ")

#     trace.span(
#             name="generation",
#             input={"question": question, "contexts": contxt, "Context_Source": context_source},
#             output={"answer": generated_answer}
#         )
    
#     # compute scores for the question, context, answer tuple
#     ragas_scores = score_with_ragas(question, contxt, generated_answer)
#     ragas_scores
#     for m in metrics:
#         trace.score(name=m.name, value=ragas_scores[m.name])
    
#     new_row = {'Question_Num': Q_Num, 'Question': query, 'Answer': generated_answer, 'Answer_Relevancy':ragas_scores['answer_relevancy'],
#               'faithfulness': ragas_scores['faithfulness'], 'context_sources': context_source, 'Contexts': contxt}
#     Q_A_DF.loc[len(Q_A_DF)] = new_row
    
#     counter+=1

In [15]:
kbs_dict = { 
    'kb1': { 'Chunk_Size': '900', 'kb_id': 'EJBFFIJDIJ', 'kb_name': 'Knowledge base name'}
}

In [18]:
KB_DF = pd.DataFrame(columns = ['Knowledge Base Name', 'Knowledge Base ID', 'Processed Date', 'Processing Time in minutes'])
for key, value in kbs_dict.items():
#     print(value['Chunk_Size'], value['kb_id'], value['kb_name'])
    kb_id = value['kb_id']
    Chunk_Size = value['Chunk_Size']
    kb_name = value['kb_name']
    Chnk_Size = "Chunk_Size: " + str(Chunk_Size)
    Max_Tokens = "Max_Tokens: " + str(MX_TKN)
    Temperature_var = "Temparature: " + str(TMPRT)
    TOP_P_var = "Top_P: " + str(TOP_P)
    TOP_K_var = 'Top_K: ' + str(TOP_K)
    
    print(f"Using Knowledge ID: {kb_id} to process duestions")
    print("=================================================")
    
    kb_start_time = time.time()
    counter = 1
    Q_A_DF = pd.DataFrame(columns = ['Question_Num', 'Question', 'Answer', 'Answer_Relevancy', 'faithfulness', 'context_sources', 'Contexts'])
    for query in questions:
        Q_Num = "Question - " + str(counter)
        print("question number:", Q_Num)
        print("Question being processed: ", query)

        question = query
        Ans = "Answer - " + str(counter)
        trace = langfuse.trace(name = kb_name, user_id = "APM_Client", 
                               tags = [Q_Num, Chnk_Size, Max_Tokens, Temperature_var, TOP_P_var, TOP_K_var])
        contexts = [docs.page_content for docs in retriever.get_relevant_documents(query)]
        trace.span(name="retrieval", 
                   input={"question": question}, 
                   output={"contexts": contexts},
            )

        # Reterieve and Generate response based on question, model_type, temperature, etc.
        response = RAG(TMPRT, MX_TKN, TOP_K, question, kb_id, region_name, '', model_id)
#         print(response)

        # Used Context for Generating Answers
        contxt = [reference['content']['text'] for citation in response['citations'] for reference in citation['retrievedReferences']]

        # Generated Answers
        generated_answer = response["output"]["text"]

        # Context Source
        context_source = [reference['location']['s3Location']['uri'] for citation in response['citations'] for reference in citation['retrievedReferences']]

    #     print(generated_answer)
    #     print(" ")

        trace.span(
                name="generation",
                input={"question": question, "contexts": contxt, "Context_Source": context_source,
                      "Max_Tokens": MX_TKN, 'Temperature': TMPRT, 'Top_P': TOP_P, 'Top_K': TOP_K},
                output={"answer": generated_answer}
            )

        # compute scores for the question, context, answer tuple
#         ragas_scores = score_with_ragas(question, contxt, generated_answer)
#         ragas_scores
#         for m in metrics:
#             trace.score(name=m.name, value=ragas_scores[m.name])

        q_a_new_row = {'Question_Num': Q_Num, 'Question': query, 'Answer': generated_answer, 
                       'context_sources': context_source, 'Contexts': contxt}
        Q_A_DF.loc[len(Q_A_DF)] = q_a_new_row

        counter+=1

        Q_A_DF.to_csv(Rslts_Save_Dir + kb_name + '_'+ MDL_TYPE + datetime.now().strftime("%Y_%m_%d") + '.csv', index=False)
        print(" ")
    
    todays_date = datetime.now().strftime("%Y_%m_%d")
    kb_end_time = time.time()
    exec_time = (kb_end_time - kb_start_time)/60
    
    kb_new_row = {'Knowledge Base Name': kb_name, 'Knowledge Base ID': kb_id, 
                  'Processed Date': todays_date, 'Processing Time in minutes': exec_time}
    KB_DF.loc[len(KB_DF)] = kb_new_row
    KB_DF.to_csv(Rslts_Save_Dir + "Chunk_Eval_Results" + '_'+ MDL_TYPE + datetime.now().strftime("%Y_%m_%d") + '.csv', index=False)

Using Knowledge ID: EJBFFIJDIJ to process duestions
question number: Question - 1
Question being processed:  What products use nylon?
 
question number: Question - 2
Question being processed:  give me a list of top 100 products that use nylon (10 is too small and is no representative)?
 
question number: Question - 3
Question being processed:  what companies use Nylon in their products?
 
question number: Question - 4
Question being processed:  who are the biggest users of nylon?
 
question number: Question - 5
Question being processed:  who is compounding Nylon in North America and South America or Latin America?
 
question number: Question - 6
Question being processed:  Are compounders using more recycled feedstock?
 
question number: Question - 7
Question being processed:  What innovations are driving the use of recycled polypropylene?
 
question number: Question - 8
Question being processed:  What are the main applications of recycled PP in the automotive industry?
 
question numbe